In [1]:
from protosc.simulation import create_categorical_data, compare_models, compare_results
from protosc.model.wrapper import Wrapper
from protosc.model.shell_all_models import execute_all_models
from protosc.model.genetic import GeneticModel
from protosc.model.utils import train_kfold_validate
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [2]:
X, y, ground_truth = create_categorical_data(min_dev=0.25, max_dev=0.5, n_categories=3, seed=1234)
print(f'ground truth #features: {len(ground_truth["selected_features"])}')
# X, y, ground_truth = create_correlated_data()

ground truth #features: 25


In [3]:
train_kfold_validate(X, y, ground_truth["selected_features"])

0.5643168043758213

In [4]:
# model = Wrapper()
# model.execute(X, y, fold_seed=102983, seed=787234, n_jobs=-1, progress_bar=True)

### Run all models

In [5]:
results = execute_all_models(X, y)

100%|██████████| 8/8 [00:15<00:00,  1.89s/it]


In [6]:
for name, res in results.items():
    print(name)
    print(compare_results(res, ground_truth))

filter
{'%corr_feat': 0.3404255319148936, '%feat_found': 0.64, '%bias_found': 0.6611111111111111}
fast_wrapper
{'%corr_feat': 0.37037037037037035, '%feat_found': 0.4, '%bias_found': 0.4011111111111111}
slow_wrapper
{'%corr_feat': 0.391304347826087, '%feat_found': 0.36, '%bias_found': 0.38666666666666666}
random
{'%corr_feat': 0, '%feat_found': 0.0, '%bias_found': 0.0}
pseudo_random
{'%corr_feat': 0, '%feat_found': 0.0, '%bias_found': 0.0}


In [7]:
gen_features = GeneticModel(n_gen_data=5, num_penalty=0.0001).execute(X, y, n_jobs=-1)

Acc: 0.51 (max: 0.58, #f: 52): 100%|██████████| 30/30 [03:01<00:00,  6.04s/it]


In [9]:
compare_results(gen_features, ground_truth)

{'%corr_feat': 0.8, '%feat_found': 0.32, '%bias_found': 0.3422222222222222}